## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lorengau,PG,-2.0226,147.2712,81.88,78,99,7.07,overcast clouds
1,1,Ayorou,NE,14.7308,0.9174,71.71,13,3,11.05,clear sky
2,2,Dudinka,RU,69.4058,86.1778,-3.39,94,100,13.87,overcast clouds
3,3,Nueve De Julio,AR,-35.4444,-60.8831,82.71,60,38,5.82,scattered clouds
4,4,Jamestown,US,42.0970,-79.2353,12.09,54,0,3.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ayorou,NE,14.7308,0.9174,71.71,13,3,11.05,clear sky
16,16,Salinopolis,BR,-0.6136,-47.3561,75.43,89,100,5.23,overcast clouds
23,23,Tambopata,PE,-12.7333,-69.1833,74.64,97,97,1.54,overcast clouds
28,28,Port Alfred,ZA,-33.5906,26.8910,70.12,85,44,12.08,scattered clouds
30,30,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,72.23,93,99,6.04,overcast clouds
33,33,Isangel,VU,-19.5500,169.2667,77.11,100,100,4.61,heavy intensity rain
40,40,Pedreira,BR,-22.7419,-46.9014,72.57,97,100,3.85,overcast clouds
41,41,Saint-Philippe,RE,-21.3585,55.7679,76.55,86,51,8.77,light rain
43,43,Mareeba,AU,-17.0000,145.4333,75.92,84,98,2.30,overcast clouds
47,47,Sao Joao Da Barra,BR,-21.6403,-41.0511,77.04,88,65,11.68,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ayorou,NE,71.71,clear sky,14.7308,0.9174,
16,Salinopolis,BR,75.43,overcast clouds,-0.6136,-47.3561,
23,Tambopata,PE,74.64,overcast clouds,-12.7333,-69.1833,
28,Port Alfred,ZA,70.12,scattered clouds,-33.5906,26.8910,
30,Sao Miguel Do Araguaia,BR,72.23,overcast clouds,-13.2750,-50.1628,
33,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,
40,Pedreira,BR,72.57,overcast clouds,-22.7419,-46.9014,
41,Saint-Philippe,RE,76.55,light rain,-21.3585,55.7679,
43,Mareeba,AU,75.92,overcast clouds,-17.0000,145.4333,
47,Sao Joao Da Barra,BR,77.04,broken clouds,-21.6403,-41.0511,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.NaN, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ayorou,NE,71.71,clear sky,14.7308,0.9174,Hotel Amenokal
16,Salinopolis,BR,75.43,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
28,Port Alfred,ZA,70.12,scattered clouds,-33.5906,26.8910,The Halyards Hotel
30,Sao Miguel Do Araguaia,BR,72.23,overcast clouds,-13.2750,-50.1628,Hotel Executivo Palace
33,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,Tanna Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))